In [ ]:

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor 

from src.filler import fill_df, interpolate_filler
from src.dataframe import show_error_metrics

In [ ]:
import sys

base_path = '/Users/yuri/git/diasYuri/tcc/radiation-forecast-mg'
sys.path.insert(0, base_path)

In [ ]:
df_temp = pd.read_csv(
    f'{base_path}/data/daily_barbacena.csv',
    sep=',',
    parse_dates=['Timestamp'],
    index_col=['Timestamp'])



df = df_temp[['RADIATION', 'TEMP', 'HUMIDITY_h']] \
    .resample('D') \
    .agg({'RADIATION': np.sum})

init_index = lambda _df: _df[_df.RADIATION != 0].index[0]
df = df.loc[df.index >= init_index(df)].replace(0, np.nan)
df

In [204]:
class Debbuger:
    @staticmethod
    def log(is_debug, *values: object):
        if is_debug:
            print(values)

class RandomForestFiller:
    model: RandomForestRegressor
    n_in: int = 5
    n_out: int = 1
    is_debug: bool = False

    def __init__(self, n_estimators):
        self.n_estimators = n_estimators

    def windowing(self, values, step_back, step_front):
        x, y = [], []
        for i in range(len(values) - step_back - step_front + 1):
            j = (i + step_back)
            x.append(values[i:j])
            y.append(values[j:(j+step_front)])
        return np.array(x), np.array(y)

    def train_test_split(self, data, n_test):
        return data[:-n_test, :], data[-n_test:, :]

    def train(self, history):
        Debbuger.log(self.is_debug, 'History:',history)
        self.model = RandomForestRegressor(n_estimators=self.n_estimators)
        trainX, trainY = self.windowing(values=history, step_back=self.n_in, step_front=self.n_out)
        Debbuger.log(self.is_debug, 'Train Data:',trainX, trainY)
        self.model.fit(trainX, trainY.ravel())
        

    def fill(self, history, retrain):
        if retrain:
            self.n_in = min((len(history)-self.n_out), 5)
            self.train(history=history)
        
        xhat = [history[-self.n_in:]]
        Debbuger.log(self.is_debug, 'xhat', xhat)
        yhat = self.model.predict(xhat)[0]
        Debbuger.log(self.is_debug, 'Predict value:', yhat)
        return yhat

def update_history(history):
    length = min(50, len(history))
    return history[-length:]

def filler(dataserie: pd.Series, filler: RandomForestFiller):
    dataserie_filled = dataserie.copy()

    history = []

    filling = False
    init_idx = None

    for i, r in enumerate(dataserie_filled):
        if pd.isna(r) or r is None:
            history = update_history(history)
            
            new_value = filler.fill(history, retrain=filling is False) 

            dataserie_filled.iloc[i] = new_value
            history.append(new_value)

            if filling:
                Debbuger.log(self.is_debug, 'completando...')
            else:
                Debbuger.log(self.is_debug, 'começando a completar...')
                filling = True
        else:
            history.append(r)
            if filling:
                Debbuger.log(self.is_debug, 'lacuna completada!')
                filling = False
    return dataserie_filled



In [205]:
print(df.isna().sum())
new_df = filler(df.RADIATION, RandomForestFiller(1000))
print(new_df.isna().sum())

RADIATION    273
dtype: int64
começando a completar...
lacuna completada!
começando a completar...
completando...
completando...
lacuna completada!
começando a completar...
lacuna completada!
começando a completar...
completando...
completando...
completando...
completando...
completando...
completando...
completando...
completando...
completando...
completando...
completando...
completando...
completando...
completando...
completando...
completando...
completando...
completando...
completando...
completando...
completando...
completando...
completando...
completando...
completando...
completando...
completando...
completando...
completando...
completando...
completando...
completando...
completando...
completando...
completando...
completando...
completando...
completando...
completando...
completando...
completando...
completando...
completando...
completando...
completando...
completando...
completando...
completando...
completando...
completando...
completando...
completando...
com